In [8]:
import os
import logging
from clickhouse_driver import Client
from dotenv import load_dotenv
from datetime import datetime, timedelta, date

# Загрузка переменных окружения
load_dotenv()

# Настройка логирования
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("update_counters.log", encoding='utf-8'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger()

# Глобальные переменные
database_name = os.getenv('CLICKHOUSE_DB', 'default')

# Подключение к ClickHouse
client = Client(
    host=os.getenv('CLICKHOUSE_HOST', '10.95.19.132'),
    user=os.getenv('CLICKHOUSE_USER', 'default'),
    password=os.getenv('CLICKHOUSE_PASSWORD', 'quie1ahpoo5Su0wohpaedae8keeph6bi'),
    port=9000,
    secure=False,
    settings={'strings_encoding': 'utf-8'}
)


def calculate_and_update_counters():
    """
    Рассчитывает и обновляет счётчики sne, ppr и repair_days в таблице OlapCube_VNV на target_date.
    Логика обновления зависит от текущего значения поля Status и предыдущего Status_P.
    """
    try:
        # Извлечь initial_date (самая ранняя дата)
        query_initial_date = f"""
        SELECT MIN(Dates) AS initial_date
        FROM {database_name}.OlapCube_VNV
        """
        initial_date_result = client.execute(query_initial_date)
        initial_date = initial_date_result[0][0]
        logger.info(f"Initial date in OlapCube_VNV: {initial_date}")

        # Проверка наличия initial_date
        if not initial_date:
            logger.error("Не удалось получить initial_date. Завершаем выполнение.")
            return

        # Вычислить target_date = initial_date + 1 день
        if isinstance(initial_date, datetime):
            initial_date_date = initial_date.date()
        elif isinstance(initial_date, date):
            initial_date_date = initial_date
        elif isinstance(initial_date, str):
            try:
                initial_date_date = datetime.strptime(initial_date, "%Y-%m-%d").date()
            except ValueError:
                logger.error(f"Неправильный формат initial_date: {initial_date}")
                return
        else:
            logger.error(f"Неизвестный формат initial_date: {initial_date}")
            return

        target_date = initial_date_date + timedelta(days=1)
        logger.info(f"Target date: {target_date}")

        # Вычислить previous_date = target_date - 1 день
        previous_date = target_date - timedelta(days=1)
        logger.info(f"Previous date: {previous_date}")

        # Получить все записи на target_date
        query_target_records = f"""
        SELECT 
            serialno, 
            Status, 
            Status_P, 
            daily_flight_hours,
            sne,
            ppr,
            repair_days
        FROM {database_name}.OlapCube_VNV
        WHERE Dates = '{target_date}'
        """
        target_records = client.execute(query_target_records)
        logger.info(f"Извлечено {len(target_records)} записей на target_date.")

        # Получить все записи на previous_date
        query_previous_records = f"""
        SELECT 
            serialno, 
            Status_P,
            sne,
            ppr,
            repair_days
        FROM {database_name}.OlapCube_VNV
        WHERE Dates = '{previous_date}'
        """
        previous_records = client.execute(query_previous_records)
        previous_records_dict = {row[0]: row[1:] for row in previous_records}
        logger.info(f"Извлечено {len(previous_records)} записей на предыдущую дату.")

        # Разделение записей по статусам
        records_by_status = {
            "Эксплуатация": [],
            "Исправен": [],
            "Ремонт": [],
            "Хранение": [],
            "Неактивно": []
        }

        unknown_statuses = set()

        for record in target_records:
            serialno, status, status_p_prev, daily_flight_hours, sne_prev, ppr_prev, repair_days_prev = record
            if status in records_by_status:
                records_by_status[status].append(record)
            else:
                unknown_statuses.add(status)

        if unknown_statuses:
            logger.warning(f"Найдены неизвестные статусы: {', '.join(unknown_statuses)}. Эти записи будут пропущены.")

        changed_serialnos = []

        # Обработка статуса "Эксплуатация"
        for record in records_by_status["Эксплуатация"]:
            serialno, status, status_p_prev, daily_flight_hours, sne_prev, ppr_prev, repair_days_prev = record

            # Получить предыдущие значения
            if serialno in previous_records_dict:
                status_p_prev_val, sne_prev_val, ppr_prev_val, repair_days_prev_val = previous_records_dict[serialno]
            else:
                logger.warning(f"Предыдущие данные не найдены для serialno: {serialno}. Используем значения по умолчанию.")
                sne_prev_val = 0.0
                ppr_prev_val = 0.0
                repair_days_prev_val = None

            # Вычислить новые значения
            sne_new = sne_prev_val + daily_flight_hours
            ppr_new = ppr_prev_val + daily_flight_hours
            repair_days_new = repair_days_prev_val  # Не изменяется

            # Проверить необходимость обновления
            if sne_new != sne_prev or ppr_new != ppr_prev:
                # Форматирование значений для SQL
                sne_new_sql = f"{sne_new}"
                ppr_new_sql = f"{ppr_new}"
                repair_days_new_sql = "NULL" if repair_days_new is None else f"{repair_days_new}"

                # Создание UPDATE запроса
                update_query = f"""
                ALTER TABLE {database_name}.OlapCube_VNV 
                UPDATE 
                    sne = {sne_new_sql},
                    ppr = {ppr_new_sql},
                    repair_days = {repair_days_new_sql}
                WHERE serialno = '{serialno}' AND Dates = '{target_date}'
                """
                client.execute(update_query)
                changed_serialnos.append(serialno)

        logger.info(f"Обработано и обновлено {len(records_by_status['Эксплуатация'])} записей статуса 'Эксплуатация'.")

        # Обработка статуса "Исправен"
        for record in records_by_status["Исправен"]:
            serialno, status, status_p_prev, daily_flight_hours, sne_prev, ppr_prev, repair_days_prev = record

            # Получить предыдущие значения
            if serialno in previous_records_dict:
                status_p_prev_val, sne_prev_val, ppr_prev_val, repair_days_prev_val = previous_records_dict[serialno]
            else:
                logger.warning(f"Предыдущие данные не найдены для serialno: {serialno}. Используем значения по умолчанию.")
                ppr_prev_val = 0.0
                repair_days_prev_val = None

            # Вычислить новые значения
            sne_new = sne_prev_val  # Не изменяется

            if status_p_prev_val == "Ремонт":
                ppr_new = 0.0
                repair_days_new = None
            else:
                ppr_new = ppr_prev_val
                repair_days_new = repair_days_prev_val  # Не изменяется

            # Проверить необходимость обновления
            needs_update = False
            if ppr_new != ppr_prev:
                needs_update = True
            if repair_days_new != repair_days_prev:
                needs_update = True

            if needs_update:
                # Форматирование значений для SQL
                sne_new_sql = f"{sne_new}"
                ppr_new_sql = f"{ppr_new}"
                repair_days_new_sql = "NULL" if repair_days_new is None else f"{repair_days_new}"

                # Создание UPDATE запроса
                update_query = f"""
                ALTER TABLE {database_name}.OlapCube_VNV 
                UPDATE 
                    sne = {sne_new_sql},
                    ppr = {ppr_new_sql},
                    repair_days = {repair_days_new_sql}
                WHERE serialno = '{serialno}' AND Dates = '{target_date}'
                """
                client.execute(update_query)
                changed_serialnos.append(serialno)

        logger.info(f"Обработано и обновлено {len(records_by_status['Исправен'])} записей статуса 'Исправен'.")

        # Обработка статуса "Ремонт"
        for record in records_by_status["Ремонт"]:
            serialno, status, status_p_prev, daily_flight_hours, sne_prev, ppr_prev, repair_days_prev = record

            # Получить предыдущие значения
            if serialno in previous_records_dict:
                status_p_prev_val, sne_prev_val, ppr_prev_val, repair_days_prev_val = previous_records_dict[serialno]
            else:
                logger.warning(f"Предыдущие данные не найдены для serialno: {serialno}. Используем значения по умолчанию.")
                repair_days_prev_val = None

            # Вычислить новые значения
            sne_new = sne_prev_val  # Не изменяется
            ppr_new = ppr_prev_val  # Не изменяется

            if status_p_prev_val == "Эксплуатация":
                repair_days_new = 1.0
            elif repair_days_prev_val is not None:
                repair_days_new = repair_days_prev_val + 1.0
            else:
                repair_days_new = 1.0

            # Проверить необходимость обновления
            if repair_days_new != repair_days_prev:
                # Форматирование значений для SQL
                sne_new_sql = f"{sne_new}"
                ppr_new_sql = f"{ppr_new}"
                repair_days_new_sql = "NULL" if repair_days_new is None else f"{repair_days_new}"

                # Создание UPDATE запроса
                update_query = f"""
                ALTER TABLE {database_name}.OlapCube_VNV 
                UPDATE 
                    sne = {sne_new_sql},
                    ppr = {ppr_new_sql},
                    repair_days = {repair_days_new_sql}
                WHERE serialno = '{serialno}' AND Dates = '{target_date}'
                """
                client.execute(update_query)
                changed_serialnos.append(serialno)

        logger.info(f"Обработано и обновлено {len(records_by_status['Ремонт'])} записей статуса 'Ремонт'.")

        # Обработка статуса "Хранение" и "Неактивно"
        for status in ["Хранение", "Неактивно"]:
            for record in records_by_status[status]:
                serialno, status_current, status_p_prev, daily_flight_hours, sne_prev, ppr_prev, repair_days_prev = record

                # Получить предыдущие значения
                if serialno in previous_records_dict:
                    _, sne_prev_val, ppr_prev_val, _ = previous_records_dict[serialno]
                else:
                    logger.warning(f"Предыдущие данные не найдены для serialno: {serialno}. Используем значения по умолчанию.")
                    sne_prev_val = 0.0
                    ppr_prev_val = 0.0

                # Вычислить новые значения (перенос с предыдущей даты)
                sne_new = sne_prev_val
                ppr_new = ppr_prev_val
                repair_days_new = repair_days_prev  # Не изменяется

                # Проверить необходимость обновления
                needs_update = False
                if sne_new != sne_prev:
                    needs_update = True
                if ppr_new != ppr_prev:
                    needs_update = True
                if repair_days_new != repair_days_prev:
                    needs_update = True

                if needs_update:
                    # Форматирование значений для SQL
                    sne_new_sql = f"{sne_new}"
                    ppr_new_sql = f"{ppr_new}"
                    repair_days_new_sql = "NULL" if repair_days_new is None else f"{repair_days_new}"

                    # Создание UPDATE запроса
                    update_query = f"""
                    ALTER TABLE {database_name}.OlapCube_VNV 
                    UPDATE 
                        sne = {sne_new_sql},
                        ppr = {ppr_new_sql},
                        repair_days = {repair_days_new_sql}
                    WHERE serialno = '{serialno}' AND Dates = '{target_date}'
                    """
                    client.execute(update_query)
                    changed_serialnos.append(serialno)

        logger.info(f"Обработано и обновлено записей статусов 'Хранение' и 'Неактивно'.")

        logger.info(f"Обновлено всего {len(changed_serialnos)} записей: sne, ppr и repair_days.")
        if changed_serialnos:
            logger.info(f"Serialnos: {', '.join(changed_serialnos)}")

    except Exception as e:
        logger.error(f"Ошибка при обновлении счётчиков: {e}")


def main():
    calculate_and_update_counters()


if __name__ == "__main__":
    main()


2024-12-01 15:22:14,171 - INFO - Initial date in OlapCube_VNV: 2024-11-25
2024-12-01 15:22:14,172 - INFO - Target date: 2024-11-26
2024-12-01 15:22:14,173 - INFO - Previous date: 2024-11-25
2024-12-01 15:22:14,246 - INFO - Извлечено 420 записей на target_date.
2024-12-01 15:22:14,295 - INFO - Извлечено 420 записей на предыдущую дату.
2024-12-01 15:22:14,297 - INFO - Обработано и обновлено 156 записей статуса 'Эксплуатация'.
2024-12-01 15:22:14,298 - INFO - Обработано и обновлено 20 записей статуса 'Исправен'.
2024-12-01 15:22:14,300 - INFO - Обработано и обновлено 8 записей статуса 'Ремонт'.
2024-12-01 15:22:15,855 - INFO - Обработано и обновлено записей статусов 'Хранение' и 'Неактивно'.
2024-12-01 15:22:15,856 - INFO - Обновлено всего 236 записей: sne, ppr и repair_days.
2024-12-01 15:22:15,858 - INFO - Serialnos: СП88В485, S1, S10, S100, S101, S102, S198, S199, S2, S20, S103, S104, S105, S106, S200, S201, S202, S203, S204, S205, S206, S207, S208, S209, S21, S210, S211, S212, S213, S